In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D


# Own utility methods:
from mandelbrot import *
from sampling_methods import *

# 1.1
We create an image of the mandelbrot set by applying the mandelbrot iteration 1000 times to a grid of complex numbers and plotting the number of iterations until divergence for each pixel.

In [ ]:
def plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as='mandelbrot_visualisation'):
    """
    Plots the Mandelbrot set divergence visualization using the computed number of divergence steps.

    Parameters:
    - num_div_steps: 2D array representing the number of steps before divergence for each point in the grid.
    - cmap: Colormap to use for the plot (default: 'viridis').
    - save_plot_as: File path to save the plot as an image.
    """
    # Plot the Mandelbrot set divergence steps
    plt.imshow(num_div_steps, cmap=cmap)
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.colorbar(label='Divergence Steps')
    plt.title('Mandelbrot Set Divergence Visualization')

    # Save the plot to the specified file path with high resolution
    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    # Display the plot
    plt.show()

# Define resolution for generating the complex grid
resolution = 1000
# Generate a complex grid over the specified range with the given resolution
C = generate_complex_grid(resolution)
# Compute the Mandelbrot set divergence steps using PyTorch implementation
num_div_steps, _ = compute_mandelbrot_torch(C, max_steps=500, bound=10)
# Plot the Mandelbrot set divergence visualization
plot_mandelbrot(num_div_steps)


We also zoom into the Mandelbrot set revealing the  intricate, self-similar patterns, enhancing visual understanding of its fractal nature.

The names of the parts we zoom into:
- Seahorse Valley
- Elephants Valley
- Spiral Region

In [ ]:
# Modify these parameters for zooming
zoom_center = (-0.75, 0.1)  # Seahorse Valley
zoom_level = 0.1  # Smaller values for deeper zooms

# Compute ranges for the zoomed-in area
real_range = (zoom_center[0] - zoom_level, zoom_center[0] + zoom_level)
imag_range = (zoom_center[1] - zoom_level, zoom_center[1] + zoom_level)

# Generate the grid with the new zoomed-in ranges
resolution = 1000
C = generate_complex_grid(resolution, real_range=real_range, imag_range=imag_range)

# Compute the Mandelbrot set with a high number of steps for detailed structure
num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=1000, bound=10)

# Plot the result
plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as= 'visualisation_seahorse')  

In [ ]:
# Modify these parameters for zooming
zoom_center = (0.3, -0.05) # Elephants Valley
zoom_level = 0.05  # Smaller values for deeper zooms 

# Compute ranges for the zoomed-in area
real_range = (zoom_center[0] - zoom_level, zoom_center[0] + zoom_level)
imag_range = (zoom_center[1] - zoom_level, zoom_center[1] + zoom_level)

# Generate the grid with the new zoomed-in ranges
resolution = 1000
C = generate_complex_grid(resolution, real_range=real_range, imag_range=imag_range)

# Compute the Mandelbrot set with a high number of steps for detailed structure
num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=1000, bound=10)

# Plot the result
plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as= 'visualisation_elephant') 

In [ ]:
# Modify these parameters for zooming
zoom_center = (-0.7, 0.3) # Spiral Region
zoom_level = 0.05  # Smaller values for deeper zooms 

# Compute ranges for the zoomed-in area
real_range = (zoom_center[0] - zoom_level, zoom_center[0] + zoom_level)
imag_range = (zoom_center[1] - zoom_level, zoom_center[1] + zoom_level)

# Generate the grid with the new zoomed-in ranges
resolution = 2000
C = generate_complex_grid(resolution, real_range=real_range, imag_range=imag_range)

# Compute the Mandelbrot set with a high number of steps for detailed structure
num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=1000, bound=10)

# Plot the result
plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as= 'visualisation_spiral') 

# 1.2
Convergence of the Area estimate with the number of iterations:

In [ ]:
def plot_mandelbrot_area_difference(sample_sizes, iteration_count, skip_iterations=0, save_plot_as='mandelbrot-convergence-iterations.png'):
    """
    Plots the difference in the estimated area of the Mandelbrot set over iterations for different sample sizes,
    illustrating how the area estimate converges as the iteration count increases.

    Parameters:
    - sample_sizes: List of sample sizes to use for the estimation.
    - iteration_count: Total number of iterations for Mandelbrot computation.
    - skip_iterations: Number of initial iterations to skip in the plot.
    - save_plot_as: File path to save the plot as an image.

    Returns:
    - area_estimates: 2D array of area estimates for each sample size and iteration.
    """
    # Initialize an array to store area estimates for each sample size and iteration count
    area_estimates = np.zeros((len(sample_sizes), iteration_count))

    # Loop over each sample size to compute area estimates
    for i, num_samples in enumerate(sample_sizes):
        # Generate complex samples using uniform random sampling method
        C = uniform_random_sampling(num_samples, (-2, 2), (-2, 2))
        # Compute Mandelbrot set area estimates using PyTorch implementation
        _, area_est = compute_mandelbrot_torch(C, iteration_count, area_factor=16)
        # Store area estimates for the current sample size
        area_estimates[i, :] = area_est

    # Create a new figure for plotting
    plt.figure(figsize=(10, 5))
    # Plot the convergence of area estimates for each sample size
    for i, samples in enumerate(sample_sizes):
        # Plot the absolute difference between the final area estimate and the estimates from each iteration
        plt.plot(np.arange(iteration_count)[skip_iterations:] + 1, np.abs(area_estimates[i, -1] - area_estimates[i, skip_iterations:]),
                 label=f's = {samples}')

    # Set plot labels and title
    plt.xlabel('Iteration j')
    plt.ylabel(r'$A_{j, s} - A_{i, s}$')
    plt.title('Convergence of Estimated Area depending on Iteration Count')
    plt.legend()
    # Set both axes to logarithmic scale for better visualization of convergence behavior
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True)
    
    # Save the plot to the specified file path with high resolution
    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    # Display the plot
    plt.show()

    return area_estimates


# Parameters used for the plot in the report (runs for over 1 hour):
# sample_sizes = [1000, 10000, 100000, 1000000]
# iteration_count = 5000000

# Example with lower run time
sample_sizes = [1000, 10000, 100000]
iteration_count = 10000

# Plot the Mandelbrot area difference for the specified parameters
plot_mandelbrot_area_difference(sample_sizes, iteration_count)


Hoeffding's inequality:

for a series of independent random variables $X_i \in [0,1]$  writing $S_n = \frac1n\sum_{i=1}^n X_i$
\begin{align}
     P(|S_n - E[S_n]| \geq \varepsilon) \leq 2 exp \left(-2\varepsilon^2 n\right) &\leq \delta\\
     {2\varepsilon^2n} &\leq -log(\delta / 2) \\
    \varepsilon &\leq \sqrt{-\frac 1{2n} log(\delta / 2)}
\end{align}

Convergence with number of samples:

In [ ]:
def plot_mandelbrot_area_difference_sample_count(sample_sizes, iteration_count, sampling_methods, method_titles, save_plot_as='images/mandelbrot-convergence-samples.png'):
    """
    Plots the difference in the estimated area of the Mandelbrot set for different sampling methods
    as a function of sample size, illustrating convergence behavior.

    Parameters:
    - sample_sizes: List of sample sizes to use for the estimation.
    - iteration_count: Number of iterations for Mandelbrot computation.
    - sampling_methods: List of sampling functions to generate complex samples.
    - method_titles: List of titles for each sampling method, used in the plot legend.
    - save_plot_as: File path to save the plot as an image.

    Returns:
    - area_estimates: 2D array of area estimates for each sampling method and sample size.
    """
    # Initialize an array to store area estimates for each method and sample size
    area_estimates = np.zeros((len(sampling_methods), len(sample_sizes)))

    # Loop over each sampling method and sample size to compute the area estimates
    for m, method in enumerate(sampling_methods):
        for i, num_samples in enumerate(sample_sizes):
            # Generate complex samples using the specified sampling method
            C = method(num_samples, (-2, 2), (-2, 2))
            # Compute Mandelbrot set area estimates using PyTorch implementation
            _, area_est = compute_mandelbrot_torch(C, iteration_count, area_factor=16)
            # Store the final area estimate for the current sampling method and sample size
            area_estimates[m, i] = area_est[-1]

    # Create a new figure for plotting
    plt.figure(figsize=(10, 5))
    
    # Define markers for each sampling method for better visualization
    markers = ['o', 'v', '^', 's', 'p']   

    # Plot the convergence of area estimates for each sampling method
    for m, method in enumerate(sampling_methods):
        # Plot the absolute difference between the current area estimate and the maximum sample size estimate
        plt.plot(sample_sizes[:-1], np.abs(area_estimates[m, :-1] - area_estimates[m, -1]),
                 label=method_titles[m], marker=markers[m])

    # Set plot labels and title
    plt.xlabel('Sample Size s')
    plt.ylabel(r'$A_{i, s} - A_{i, s_{max}}$')
    plt.title('Convergence of Estimated Area depending on Sample Size')
    plt.legend()
    # Set both axes to logarithmic scale for better visualization of convergence behavior
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True)
    
    # Save the plot to the specified file path with high resolution
    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    # Display the plot
    plt.show()

    return area_estimates

# Define sample sizes, iteration count, and sampling methods for comparison
sample_sizes = np.logspace(3, 7, 20, base=10).astype(np.int64)
iteration_count = 10000
sampling_methods = [uniform_random_sampling, latin_hypercube_sampling, orthogonal_sampling]
method_titles = ['uniform random', 'latin hypercube', 'orthogonal']

# Plot the area difference and get the area estimates
area_estimates = plot_mandelbrot_area_difference_sample_count(sample_sizes, iteration_count, sampling_methods, method_titles)
print(area_estimates)


# 1.3
In this part we look at different sampling methods to estimate the size of the Mandelbrot set to compare.


 *Pure Random Sampling*

In [ ]:
def plot_sampling_method(C, title, max_steps=1000, area_factor=9, save_plot_as = 'sampling_method'):
    """
    Computes and plots Mandelbrot set divergence steps for a given complex array using a specified sampling method.

    Parameters:
    - C: 1D array of complex numbers representing the samples.
    - title: Title for the plot.
    - max_steps: Maximum number of iterations for Mandelbrot computation.
    - area_factor: Area factor for the computation.
    """
    num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=max_steps, area_factor=area_factor)

    rval, ival = C.real, C.imag
    plt.scatter(rval, ival, s=0.1, alpha=0.1, c=num_div_steps)
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.title(title)
    plt.colorbar(label='Divergence Steps')

    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    plt.show()

def pure_random_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=42):
    """
    Performs pure random sampling for the Mandelbrot set.

    Parameters:
    - num_samples: Number of samples.
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - C: 1D array of complex numbers representing the samples.
    """
    np.random.seed(seed)
    rval = np.random.uniform(real_range[0], real_range[1], num_samples)
    ival = np.random.uniform(imag_range[0], imag_range[1], num_samples)
    return rval + 1.j * ival

# Example Usage
C = pure_random_sampling(num_samples=100000)
plot_sampling_method(C, "Mandelbrot Set Divergence Steps (Pure Random Sampling)", save_plot_as = 'pure_random_sampling')

*Orthogonal Sampling*

In [ ]:
def orthogonal_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=42):
    """
    Performs orthogonal sampling for the Mandelbrot set, suited for Python while following the logic of the provided C code.

    Parameters:
    - num_samples: Total number of samples to generate (must be a perfect square).
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - C: 1D array of complex numbers representing the samples.
    """
    # Ensure num_samples is a perfect square for orthogonal grid
    np.random.seed(seed)
    major = int(np.round(np.sqrt(num_samples)))
    num_samples = major * major
    print(f"Adjusted num_samples to {num_samples} to ensure it is a perfect square close to the original value.")

    x_indices = np.arange(major)
    y_indices = np.arange(major)

    samples = []
    for i in range(major):
        # Shuffle indices for randomized sampling
        np.random.shuffle(x_indices)
        np.random.shuffle(y_indices)

        for j in range(major):
            # Generate random perturbations within each grid cell
            rand_real = np.random.uniform(0, 1)
            rand_imag = np.random.uniform(0, 1)

            # Map to the real and imaginary ranges
            x = real_range[0] + (real_range[1] - real_range[0]) * ((i + rand_real) / major)
            y = imag_range[0] + (imag_range[1] - imag_range[0]) * ((j + rand_imag) / major)

            samples.append(complex(x, y))

    return np.array(samples)


# Example Usage
C = orthogonal_sampling(num_samples=100000)
plot_sampling_method(C, "Mandelbrot Set Divergence Steps (Orthogonal Sampling)", save_plot_as='orthogonal_sampling')

*Latin HyperCube Sampling*

In [ ]:
def latin_hypercube_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=42):
    """
    Performs Latin Hypercube sampling for the Mandelbrot set.

    Parameters:
    - num_samples: Number of samples.
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - C: 1D array of complex numbers representing the samples.
    """
    # Generate Latin Hypercube samples
    np.random.seed(seed)
    sampler = LatinHypercube(d=2)
    lhs_samples = sampler.random(n=num_samples)

    # Scale samples to the desired ranges
    scaled_samples = scale(lhs_samples, [real_range[0], imag_range[0]], [real_range[1], imag_range[1]])
    rval, ival = scaled_samples[:, 0], scaled_samples[:, 1]
    return rval + 1.j * ival

# Example Usage
C = latin_hypercube_sampling(num_samples=100000)
plot_sampling_method(C, "Mandelbrot Set Divergence Steps (Latin Hypercube Sampling)", save_plot_as = 'latin_hypercube_sampling')

# 1.4

In [ ]:
def is_in_mandelbrot(c, max_iter):
    """
    Determines if a complex number is in the Mandelbrot set.

    Parameters:
    - c: Complex number.
    - max_iter: Maximum number of iterations to check.

    Returns:
    - Boolean indicating whether the point is in the Mandelbrot set.
    """
    z = 0
    for n in range(max_iter):
        z = z*z + c
        if abs(z) > 2:
            return False
    return True

def estimate_area_mandelbrot(num_samples, max_iter, real_range=(-3, 1), imag_range=(-2, 2)):
    """
    Estimates the area of the Mandelbrot set using Monte Carlo sampling.

    Parameters:
    - num_samples: Number of random samples.
    - max_iter: Maximum number of iterations for Mandelbrot check.
    - real_range: Range of real values (tuple).
    - imag_range: Range of imaginary values (tuple).

    Returns:
    - Estimated area of the Mandelbrot set.
    """
    real = np.random.uniform(real_range[0], real_range[1], num_samples)
    imag = np.random.uniform(imag_range[0], imag_range[1], num_samples)
    complex_samples = real + 1j * imag

    in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])

    rect_area = (real_range[1] - real_range[0]) * (imag_range[1] - imag_range[0])
    area_estimate = np.mean(in_set) * rect_area
    return area_estimate

### Adaptive sampling

Here we first go through two extra methods of sampling. We did not end up using these, but they are left in here for completeness.

In [ ]:
true_area = 1.5063

# Function for Importance Sampling
def importance_sampling(num_samples, max_iter, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=None):
    """
    Performs importance sampling for estimating the area of the Mandelbrot set.

    Parameters:
    - num_samples: Number of samples.
    - max_iter: Maximum number of iterations for Mandelbrot membership.
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - weighted_area_estimate: Estimated area of the Mandelbrot set.
    """
    if seed is None:
        seed = random.randint(0, 10000)
    np.random.seed(seed)

    # Generate normally distributed samples
    mean_real = 0
    mean_imag = 0
    std_dev = 0.8
    
    real_samples = np.random.normal(mean_real, std_dev, num_samples)
    imag_samples = np.random.normal(mean_imag, std_dev, num_samples)

    # Clip samples to remain within specified ranges
    real_samples = np.clip(real_samples, real_range[0], real_range[1])
    imag_samples = np.clip(imag_samples, imag_range[0], imag_range[1])

    # Combine into complex numbers
    complex_samples = real_samples + 1j * imag_samples

    # Compute proposal and target densities
    proposal_density = (1 / (2 * np.pi * std_dev**2)) * np.exp(-(real_samples**2 + imag_samples**2) / (2 * std_dev**2))
    target_density = 1 / ((real_range[1] - real_range[0]) * (imag_range[1] - imag_range[0]))

    # Determine if samples are in the Mandelbrot set
    in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])

    # Compute weights and estimate area
    weights = target_density / proposal_density
    rect_area = (real_range[1] - real_range[0]) * (imag_range[1] - imag_range[0])
    weighted_area_estimate = np.mean(in_set * weights) * rect_area

    return weighted_area_estimate

# Function for Sobol Sampling (Quasi-Monte Carlo)
def sobol_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=None):
    """
    Performs Sobol sampling (Quasi-Monte Carlo) for the Mandelbrot set.

    Parameters:
    - num_samples: Number of samples.
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - C: 1D array of complex numbers representing the samples.
    """
    if seed is None:
        seed = random.randint(0, 10000)
    np.random.seed(seed)

    # Generate Sobol samples
    sampler = Sobol(d=2, scramble=True)
    sobol_samples = sampler.random(n=num_samples)

    # Scale samples to the desired ranges
    scaled_samples = scale(sobol_samples, [real_range[0], imag_range[0]], [real_range[1], imag_range[1]])

    rval, ival = scaled_samples[:, 0], scaled_samples[:, 1]
    return rval + 1.j * ival

Here, is our actual used method: Adaptive Sampling.

In [ ]:
# Refactored AdaptiveSampler class
class AdaptiveSampler:
    def __init__(self, num_samples, max_iter, real_range=(-2, 1), imag_range=(-1.5, 1.5),
                 seed=None, error_threshold=1e-4, max_iterations=10):
        """
        Initializes the AdaptiveSampler class with specified parameters.

        Parameters:
        - num_samples: Number of samples to generate.
        - max_iter: Maximum number of iterations for Mandelbrot computation.
        - real_range: Tuple specifying the range for the real axis.
        - imag_range: Tuple specifying the range for the imaginary axis.
        - seed: Random seed for reproducibility.
        - error_threshold: Threshold for stopping based on variance.
        - max_iterations: Maximum number of iterations for adaptive refinement.
        """
        self.num_samples = num_samples
        self.max_iter = max_iter
        self.real_range = real_range
        self.imag_range = imag_range
        self.seed = seed if seed is not None else random.randint(0, 10000)
        self.error_threshold = error_threshold
        self.max_iterations = max_iterations

        # Set random seed for reproducibility
        np.random.seed(self.seed)

        # Determine initial grid size and samples per region
        self.initial_grid_size = max(1, int(np.sqrt(self.num_samples)))
        self.initial_samples_per_region = max(1, self.num_samples // (self.initial_grid_size ** 2))
        self.regions = self.create_grid()

    class Region:
        def __init__(self, x_min, x_max, y_min, y_max):
            """
            Initializes a region with specified boundaries.

            Parameters:
            - x_min, x_max: Boundaries for the real axis.
            - y_min, y_max: Boundaries for the imaginary axis.
            """
            self.x_min = x_min
            self.x_max = x_max
            self.y_min = y_min
            self.y_max = y_max
            self.samples = []
            self.in_set_counts = []
            self.total_samples = 0
            self.local_area_estimate = 0
            self.local_variance = np.inf

        def sample(self, num_samples, max_iter):
            """
            Samples points within the region and updates area estimate and variance.

            Parameters:
            - num_samples: Number of points to sample.
            - max_iter: Maximum number of iterations for Mandelbrot computation.
            """
            x_samples = np.random.uniform(self.x_min, self.x_max, num_samples)
            y_samples = np.random.uniform(self.y_min, self.y_max, num_samples)
            complex_samples = x_samples + 1j * y_samples
            in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])
            self.samples.extend(complex_samples)
            self.in_set_counts.extend(in_set)
            self.total_samples += num_samples

            # Calculate area estimate and variance for the region
            area = (self.x_max - self.x_min) * (self.y_max - self.y_min)
            p = np.mean(self.in_set_counts)
            self.local_area_estimate = p * area
            if self.total_samples > 0:
                self.local_variance = (p * (1 - p) / self.total_samples) * area ** 2
            else:
                self.local_variance = np.inf

        def increase_sample_count(self, num_samples, max_iter):
            """
            Increases the number of samples in the region for further refinement.

            Parameters:
            - num_samples: Additional number of points to sample.
            - max_iter: Maximum number of iterations for Mandelbrot computation.
            """
            self.sample(num_samples, max_iter)

    def create_grid(self):
        """
        Creates an initial grid of regions based on the specified real and imaginary ranges.

        Returns:
        - regions: List of Region objects representing the initial grid.
        """
        x_min, x_max = self.real_range
        y_min, y_max = self.imag_range
        x_edges = np.linspace(x_min, x_max, self.initial_grid_size + 1)
        y_edges = np.linspace(y_min, y_max, self.initial_grid_size + 1)
        regions = []
        for i in range(self.initial_grid_size):
            for j in range(self.initial_grid_size):
                region = self.Region(x_edges[i], x_edges[i + 1], y_edges[j], y_edges[j + 1])
                regions.append(region)
        return regions

    def combine_region_estimates(self):
        """
        Combines area estimates from all regions to provide a total area estimate.

        Returns:
        - total_area_estimate: Combined area estimate from all regions.
        """
        total_area_estimate = sum(region.local_area_estimate for region in self.regions)
        return total_area_estimate

    def estimate_overall_variance(self):
        """
        Estimates the overall variance across all regions.

        Returns:
        - total_variance: Sum of variances from all regions.
        """
        total_variance = sum(region.local_variance for region in self.regions)
        return total_variance

    def select_regions_with_high_variance(self, fraction=0.5):
        """
        Selects regions with the highest variance for further sampling.

        Parameters:
        - fraction: Fraction of regions to select based on variance.

        Returns:
        - regions_to_refine: List of regions with high variance.
        """
        variances = np.array([region.local_variance for region in self.regions])
        threshold = np.percentile(variances, 100 * (1 - fraction))
        regions_to_refine = [region for region in self.regions if region.local_variance >= threshold]
        return regions_to_refine

    def run(self):
        """
        Runs the adaptive sampling process to estimate the Mandelbrot set area.

        Returns:
        - area_estimate: Final area estimate after adaptive sampling.
        """
        for iteration in range(self.max_iterations):
            # Sample in each region
            for region in self.regions:
                if region.total_samples == 0:
                    samples_to_draw = self.initial_samples_per_region
                else:
                    samples_to_draw = max(1, self.initial_samples_per_region // (iteration + 1))
                region.sample(samples_to_draw, self.max_iter)
            # Combine estimates
            area_estimate = self.combine_region_estimates()
            # Estimate variance
            variance_estimate = self.estimate_overall_variance()
            # Check for convergence
            if variance_estimate < self.error_threshold:
                break
            # Select regions to refine
            regions_to_refine = self.select_regions_with_high_variance()
            if not regions_to_refine:
                break
            # Increase sampling in selected regions
            additional_samples = max(1, self.initial_samples_per_region // (iteration + 1))
            for region in regions_to_refine:
                region.increase_sample_count(additional_samples, self.max_iter)
        return area_estimate


def run_simulations(sample_sizes, max_iter=1000):
    """
    Runs simulations for different sampling methods and compares errors.

    Parameters:
    - sample_sizes: List of sample sizes to use.
    - max_iter: Maximum number of iterations for Mandelbrot computation.

    Returns:
    - errors_random: List of errors for random sampling.
    - errors_orthogonal: List of errors for orthogonal sampling.
    - errors_adaptive: List of errors for adaptive sampling.
    """
    errors_random = []
    errors_adaptive = []
    errors_orthogonal = []

    for num_samples in sample_sizes:
        seed = random.randint(0, 10000)

        # Normal Monte Carlo (Random Sampling)
        C_random = pure_random_sampling(num_samples, seed=seed)
        area_random = estimate_area_mandelbrot(C_random, max_iter)
        error_random = abs(area_random - true_area)
        errors_random.append(error_random)

        # Orthogonal Sampling
        C_orthogonal = orthogonal_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=seed)
        area_orthogonal = estimate_area_mandelbrot(C_orthogonal, max_iter)
        error_orthogonal = abs(area_orthogonal - true_area)
        errors_orthogonal.append(error_orthogonal)

        # Adaptive Sampling
        adaptive_sampler = AdaptiveSampler(num_samples, max_iter, seed=seed)
        area_adaptive = adaptive_sampler.run()
        error_adaptive = abs(area_adaptive - true_area)
        errors_adaptive.append(error_adaptive)

    return errors_random, errors_orthogonal, errors_adaptive

# Set up sample sizes for the simulations
min_sample_size = 1000
max_sample_size = 1000000
sample_sizes = np.logspace(np.log10(min_sample_size), np.log10(max_sample_size), num=29).astype(int)

# Run simulations and gather errors
errors_random, errors_orthogonal, errors_adaptive = run_simulations(sample_sizes)

# Calculate mean and standard deviation for orthogonal and adaptive sampling
mean_orthogonal = np.mean(errors_orthogonal)
std_orthogonal = np.std(errors_orthogonal)
mean_adaptive = np.mean(errors_adaptive)
std_adaptive = np.std(errors_adaptive)

# Statistical comparison between orthogonal and adaptive sampling
t_stat, p_value = stats.ttest_ind(errors_orthogonal, errors_adaptive)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

# Print summary of statistical analysis
print("Statistical Summary:")
print(f"Orthogonal Sampling: Mean = {mean_orthogonal:.4f}, Std Dev = {std_orthogonal:.4f}")
print(f"Adaptive Sampling: Mean = {mean_adaptive:.4f}, Std Dev = {std_adaptive:.4f}")
print("\nT-tests Results:")
print(f"Orthogonal vs Adaptive: t-statistic = {t_stat:.4f}, p-value = {p_value:.4g}")

# Plotting the results to visualize convergence rates
plt.figure(figsize=(8, 6))
plt.loglog(sample_sizes, errors_adaptive, marker='o', label="Adaptive Sampling")
plt.loglog(sample_sizes, errors_orthogonal, marker='o', label="Orthogonal Sampling")
plt.xlabel('Number of Samples')
plt.ylabel('Error |A_est - A_true|')
plt.title('Comparison of Convergence Rates for Monte Carlo Methods')
plt.legend()
plt.grid(True)
plt.show()
